In [26]:
import sys
import os

# Agregar la ruta del directorio scripts
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

### Extract and Transform

Se convirtio el dataset `movies_dataset.csv` a formato `parquet` con las siguientes modificaciones:

- **Columnas eliminadas**: `video`, `imdb_id`, `adult`, `original_title`, `poster_path`, `homepage`
- **Valores nulos rellenados**:
  - `revenue`: 0
  - `budget`: 0
- **Conversión de tipos**:
  - `popularity`: `float64`
  - `budget`: `int64`
  - `id`: `int64`

En la carpeta `scripts/convert_csv_to_parquet.py` puedes ver el funcionamiento de la funcion.
- Una vez transformado se recomienda eliminar el archivo .csv 

In [28]:
from convert_csv_to_parquet import convertir_csv_a_parquet

convertir_csv_a_parquet(
    r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\raw\movies_dataset.csv', # Origen
    r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\raw\movies_dataset.parquet', # Destino
    columns_to_drop=['video','imdb_id','adult','original_title','poster_path','homepage'],
    fillna_values={'revenue': 0, 'budget': 0},
    dtype_conversion={'popularity': 'float64', 'budget':'int64','id':'int64'}
)

c:\Users\mauri\OneDrive\Escritorio\MLops\scripts\convert_csv_to_parquet.py:33: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [29]:
import pandas as pd

df = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\raw\movies_dataset.parquet')
df.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0
1,None,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0


Eliminar valores nulos de `release_date`

In [30]:
df = df.dropna(subset=['release_date'])

formato `AAAA-mm-dd` en fechas, y creacion de columna `release_year` para el año de estreno.

In [31]:
# Identificar las filas con valores incorrectos
incorrect_format = df[~df['release_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]
# Muestra las filas con formato incorrecto.
incorrect_format

# Ya que los errores no pueden ser corregidos manualmente por falta de informacion
# Y ademas las demas filas parecieran incorrectas y con muchos Nan. Se procede a eliminar estos datos.

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
19730,0.065736,NaN,"[{'name': 'Carousel Productions', 'id': 11176}...",NaN,104.0,Released,NaN,False,6.0,1,0.0,NaN,None,None,None,None,NaN,NaN
29503,1.931659,NaN,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",NaN,68.0,Released,NaN,False,7.0,12,0.0,NaN,None,None,None,None,NaN,NaN
35587,2.185485,NaN,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...",NaN,82.0,Released,NaN,False,4.3,22,0.0,NaN,None,None,None,None,NaN,NaN


In [32]:
# Se borraron esas 3 filas del df.
df.drop(incorrect_format.index, inplace=True)

In [33]:
# Ahora si pasamos a datetime con formato aaaa-mm-dd
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d')

# Creacion columna release_year y muestra. 
df['release_year'] = df['release_date'].dt.year
df[['release_date','release_year']].sample(5)


,release_date,release_year
3661,1998-06-25,1998
7551,2002-12-12,2002
24256,2014-09-19,2014
3333,2000-03-23,2000
22994,2013-09-06,2013


crear columna  `return` dividiendo  `revenue` / `budget`, cuando no hay datos para calcularlo, tomar valor 0.



In [34]:
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if pd.notnull(row['revenue']) and pd.notnull(row['budget']) and row['budget'] != 0 else 0, axis=1)

- `belongs_to_collection `,  `production_companies ` ,  `genres ` ,  `production_countries`,  `spoken_languages` están anidados.
- deberán desanidarlos O buscar la manera de acceder sin desanidarlos.

### Empecemos por `belongs_to_collection `

In [35]:
# Esta columna contiene 40888 datos nulos y sus formatos en str . 
df['belongs_to_collection'].apply(type).value_counts() 

belongs_to_collection
<class 'NoneType'>    40888
<class 'str'>          4488
Name: count, dtype: int64

In [36]:
# Por lo tanto crearemos una nueva columna que muestre su tipo de dato
df['belongs_to_collection_type'] = df['belongs_to_collection'].apply(lambda x: type(x).__name__)

# Podemos observar que estan anidados como string, en vez de dict, esto dificulta saber si son strings o diccionarios realmente para su transformacion
df[['belongs_to_collection_type','belongs_to_collection']].head(4)

,belongs_to_collection_type,belongs_to_collection
0,str,"{'id': 10194, 'name': 'Toy Story Collection', ..."
1,NoneType,None
2,str,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
3,NoneType,None


In [37]:
import ast

# Función para convertir  str en diccionarios en caso de serlo.
def convert_dict(val: any) -> dict | str | None:
    """
    Convierte un posible diccionario con (type:str), A su forma verdadera (type:dict).
    Si la fila de entrada es NaN, devuelve None. Si el valor ya es un diccionario, lo devuelve
    sin cambios. Para entradas str, intenta analizarlas como diccionario. Si no es un diccionario ni un nulo, devuelve su forma original de str.

    Parameters:
    --------
    val: Any
        El valor a verificar y convertir, puede ser str, dict o Nan.

    Returns:
    --------
    dict |  str | None:
        dict si la conversión es exitosa, str si no puede convertirse en diccionario, None si ya era nulo.
    """
    
    
    if pd.isna(val):
        return None

    if isinstance(val, str): 
        try:
            # Reemplazar 'Null' por 'None' para que sea un dict válido
            val = val.replace('Null', 'None')

            # Asegurarse de que el valor empiece y termine con { }
            if val.startswith('{') and val.endswith('}'):
                # Evaluar el str como un diccionario usando ast.literal_eval
                return ast.literal_eval(val)
            
            # Si el valor no es un dict válido, devolver la cadena original
            return val
        except (ValueError, SyntaxError) as e:
            print(f"Error convirtiendo el valor: {val}\nError: {e}")
            return val
    
    return val  # Si ya es un diccionario, lo devolvemos sin cambiar


df['belongs_to_collection'] = df['belongs_to_collection'].apply(convert_dict)

df['belongs_to_collection_type'] = df['belongs_to_collection'].apply(lambda x: type(x).__name__)

# Mostrar el DataFrame con los tipos corregidos
df[[ 'belongs_to_collection', 'belongs_to_collection_type']].head(4)

,belongs_to_collection,belongs_to_collection_type
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",dict
1,None,NoneType
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",dict
3,None,NoneType


In [38]:
# Ahora si podemos verificar que no hay strings escondidos o erroneos y que todos son diccionarios en realidad.
df['belongs_to_collection'].apply(type).value_counts() 

belongs_to_collection
<class 'NoneType'>    40888
<class 'dict'>         4488
Name: count, dtype: int64

In [39]:
# Desanidar los diccionarios en la columna 'belongs_to_collection'
belong_desanidado = pd.json_normalize(df['belongs_to_collection'])

# Renombrar las columnas agregando el prefijo 'btc_'
belong_desanidado.rename(columns={'id': 'btc_id',
                                  'name':'btc_name',
                                  'poster_path':'btc_poster_path',
                                  'backdrop_path': 'btc_backdrop_path'}, inplace=True)
belong_desanidado.head(4)

,btc_id,btc_name,btc_poster_path,btc_backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,NaN,NaN,NaN,NaN
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,NaN,NaN,NaN,NaN


In [40]:
df = pd.concat([df, belong_desanidado], axis=1)

# Ya podemos borrar la columna auxiliar, belongs_to_collection_type
df = df.drop(columns=['belongs_to_collection_type'])

### Desanidemos las demas columnas anidadas. `genres`. Que al ser una lista de diccionarios, tiene un procedimiento distinto.

In [41]:
# La funcion convert_dict() no funcionaria en esta columna. ya que son listas de diccionarios y no un solo diccionario.

import json
import pandas as pd

# Convertir posibles listas de diccionarios categorizadas como str en listas
def convert_listdict(val: any) -> list | str | None:
    """
    Convierte una posible lista de diccionarios en formato (type:str) en una lista de diccionarios (type:list).
    Si la entrada es NaN, devuelve None. Si el valor ya es una lista, lo devuelve sin cambios.
    Para entradas str, intenta analizarla como JSON después de reemplazar comillas simples por comillas dobles.
    Si la cadena no es una lista válida, devuelve su forma original de str.

    Parameters:
    --------
    val: Any
        El valor a verificar y convertir, puede ser str, list o Nan.

    Returns:
    --------
    list | str | None:
        list si la conversión es exitosa, str si no puede convertirse en lista, None si ya era nulo.
    """
    
    if pd.isna(val):
        return None

    if isinstance(val, str):
        try:
            # Reemplazar comillas simples por dobles para un JSON válido
            val = val.replace("'", '"')
            # Convertir la cadena a un objeto JSON (lista de diccionarios)
            converted = json.loads(val)
            if isinstance(converted, list) and all(isinstance(item, dict) for item in converted):
                return converted
            else:
                # Si el JSON no es una lista de diccionarios, devolver la cadena original
                return val
        except (ValueError, json.JSONDecodeError) as e:
            #print(f"Error convirtiendo el valor: {val}\nError: {e}")
            return val
    
    return val  # Si ya es una lista, lo devolvemos sin cambiar

# Aplicar la función a la columna 'genres'
df['genres'] = df['genres'].apply(convert_listdict)

In [42]:
# Mas del 99% del dataset fue transformado exitosamente.
df['genres'].apply(type).value_counts()

genres
<class 'list'>        45376
<class 'NoneType'>       88
Name: count, dtype: int64

Como estos son listas de diccionarios y no un solo diccionario con unico valor para cada fila, la logica es que existan multiples valores `id` y `names` para cada fila.
Por lo tanto voy a usar lo logica de joins de SQL. Poniendo solo un identificador unico para cada fila de `genres` llamado `genres_id` en el dataframe orignal, pero que ese `genres_id` pueda compartir multiples valores en la tabla `genres_desanidado`. que luego puede ser mergeada o concatenada para saber sus multiples valores. 

In [43]:
import pandas as pd
from pandas import DataFrame

def desanidar_columnas(df:DataFrame,
                       columna:str,
                       nombre_nueva_tabla: str,
                       indice_increment:int=0) -> DataFrame:
    """
    Desanida una columna en un DataFrame, genera un ID único que relaciona
    la tabla original con nombre_nueva_tabla desanidada.
    Antes de aplicar esta funcion tienes que aplicar la funcion convert_listdict() , o la funcion convert_dict() segun su caso.
    o verificar que el tipo de dato de la columna ya este transformado.

    Parameters:
    ----------
    df (DataFrame):
        DataFrame original, donde se insertara el indice.
    columna (str):
        Nombre de la columna del df a desanidar. Verificar que el tipo de dato este en list, o dict.
    nombre_nueva_tabla (str):
        Nombre de la tabla donde se pondran los datos desanidados.
    nombre_exploded (str):
        Nombre de la tabla donde se pondran los datos de la funcion exlpoded.
    indice_increment (int):
        Valor opcional a sumar al índice para crear el id único.

    Return:
    --------
    nombre_nueva_tabla (DataFrame): Nuevo DataFrame desanidado, con el nombre asignado
    """
    df = df
    
    # Crear el ID único en el DataFrame original
    df[f'{columna}_id'] = df.index + indice_increment

    # Explode para desanidar la columna de listas
    df_exploded = df.explode(columna)

    # Normalizar los datos y creacion de la columna desanidada
    nombre_nueva_tabla = pd.json_normalize(df_exploded[columna])
    # Asignar el ID único para relacionar las tablas
    nombre_nueva_tabla[f'{columna}_id'] = df_exploded[f'{columna}_id'].values

    return nombre_nueva_tabla


In [44]:
# Aplicamos la funcion y ahora tenemos la tabla desanidada y la tabla original con los id pertenecientes de la tabla desanidada.
#Le asignamos el nombre que en realidad es un Dataframe a una variable.
genres_desanidado = desanidar_columnas(df, 'genres', 'genres_desanidado')

genres_desanidado.head(4)

,id,name,genres_id
0,16.0,Animation,0
1,35.0,Comedy,0
2,10751.0,Family,0
3,12.0,Adventure,1


### Sigamos desanidado columnas anidadas. Ahora: `production_companies`.

In [45]:
# Vemos que la mayoria de los valores son listas de diccionarios mal tipeados. y que contiene 88 floats.
df['production_companies'].apply(type).value_counts()

production_companies
<class 'str'>      45376
<class 'float'>       88
Name: count, dtype: int64

In [46]:
# Al verificar los floats, vemos que contiene nulos en todas las columnas. Por ende eliminaremos estas filas.
# pd.set_option('display.max_rows', None) # Con este codigo verificamos las 88 filas. Descomentar para verificar.
df[df['production_companies'].apply(lambda x: isinstance(x, float))].head(5)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,title,vote_average,vote_count,release_year,return,btc_id,btc_name,btc_poster_path,btc_backdrop_path,genres_id
711,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,711
734,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,734
3460,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3460
3628,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3628
5879,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5879


In [47]:
#Eliminar esos floats nulos.
df = df.dropna(subset=['production_companies'])

In [48]:
# Podemos aplicar la misma funcion que usamos en 'genres' ya que aqui tambien son listas de diccionarios.
df['production_companies'] = df['production_companies'].apply(convert_listdict)
df['production_companies'].apply(type).value_counts()

# No es 100% exitosa por problemas de comillas dobles pero mas del 98% del dataset fue transformado con exito.

production_companies
<class 'list'>    44892
<class 'str'>       484
Name: count, dtype: int64

In [49]:
# Aplicamos la funcion que desanida las columnas en otra tabla. 
# 1 es para que no sean igual los id a genres_id
# Lo asignamos como una variable que es ahi donde estaran las tablas
pc_desanidado = desanidar_columnas(df, 'production_companies', 'pc_desanidado', 1)

pc_desanidado.head(4)

,name,id,production_companies_id
0,Pixar Animation Studios,3.0,1
1,TriStar Pictures,559.0,2
2,Teitler Film,2550.0,2
3,Interscope Communications,10201.0,2


### Sigamos desanidando. Ahora: `production_countries`. Que contiene el mismo formato de datos que production_companies.

In [50]:
# Seguimos aplicando la misma funcion. Con mas de 99% de exito. 
df['production_countries'] = df['production_countries'].apply(convert_listdict)
df['production_countries'].apply(type).value_counts()

production_countries
<class 'list'>    45371
<class 'str'>         5
Name: count, dtype: int64

In [51]:
# Aplicamos la funcion que desanida las columnas en otra tabla. 
# Lo asignamos como una variable que es ahi donde estaran las tablas
pctry_desanidado = desanidar_columnas(df, 'production_countries', 'pctry_desanidado', 2)

pctry_desanidado.head(4)

,iso_3166_1,name,production_countries_id
0,US,United States of America,2
1,US,United States of America,3
2,US,United States of America,4
3,US,United States of America,5


### Desanidemos `spoken_languages`. Otra lista de diccionarios. Que repite el mismo procedimiento.

In [52]:
df['spoken_languages'] = df['spoken_languages'].apply(convert_listdict)
df['spoken_languages'].apply(type).value_counts()

spoken_languages
<class 'list'>    45350
<class 'str'>        26
Name: count, dtype: int64

In [53]:
# Aplicamos la funcion que desanida las columnas en otra tabla. 
# Lo asignamos como una variable que es ahi donde estaran las tablas
slan_desanidado = desanidar_columnas(df, 'spoken_languages', 'slan_desanidado', 3)

slan_desanidado.head(4)


,iso_639_1,name,spoken_languages_id
0,en,English,3
1,en,English,4
2,fr,Français,4
3,en,English,5


Modificaciones necesarias para la API

In [88]:
# Cambiamos a minisculas las peliculas para que la api no diferencie entre minusculas y mayusculas los titulos.
df['title'] = df['title'].str.lower()

# columnas tipo float. cambiemoslo a int.
df['release_year'] = df['release_year'].astype(int)
df['vote_count'] = df['vote_count'].astype(int)

In [92]:
df[['title','vote_count']]

,title,vote_count
0,toy story,5415
1,jumanji,2413
2,grumpier old men,92
3,waiting to exhale,34
4,father of the bride part ii,173
...,...,...
45371,robin hood,26
45372,century of birthing,3
45373,betrayal,6
45374,satan triumphant,0


Hemos terminado movies_dataset. Asi ha quedado, luego de las transformaciones:

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   belongs_to_collection    4488 non-null   object        
 1   budget                   45376 non-null  float64       
 2   genres                   45376 non-null  object        
 3   id                       45376 non-null  float64       
 4   original_language        45365 non-null  object        
 5   overview                 44435 non-null  object        
 6   popularity               45376 non-null  float64       
 7   production_companies     45376 non-null  object        
 8   production_countries     45376 non-null  object        
 9   release_date             45376 non-null  datetime64[ns]
 10  revenue                  45376 non-null  float64       
 11  runtime                  45130 non-null  float64       
 12  spoken_languages         45376 n

Tambien hemos creado tablas extras:
- `genres_desanidado`
- `pc_desanidado`
- `pctry_desanidado`
- `slan_desanidado`

que exportaremos a junto con el dataset principal, a la carpeta processed. Ya que ya esta listo para pasar al eda. 

- Como ya hemos desanidado las listas. volveremos a pasarlas a str, para poder exportarlas.

In [61]:
def lista_a_str(val):
    if isinstance(val, list):
        return str(val)
    else:
        return val

# Aplicar la función de conversión a la columna
df['production_companies'] = df['production_companies'].apply(lista_a_str)
df['production_countries'] = df['production_countries'].apply(lista_a_str)
df['spoken_languages'] = df['spoken_languages'].apply(lista_a_str)

Exportamos el df principal.

In [90]:
# Asegúrate de que el directorio de destino exista
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

# Ruta y nombre del archivo a guardar.
parquet_path = os.path.join(output_dir, 'movies_dataset_etl.parquet')
# Guardar el DataFrame en la carpeta data/processed
df.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a 'data/processed'")

Datos exportados correctamente a 'data/processed'


Exportamos las tablas extras.

In [64]:
# Ruta y nombre del archivo a guardar.
parquet_path = os.path.join(output_dir, 'genres_desanidado.parquet')
# Guardar el DataFrame en la carpeta data/processed
genres_desanidado.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a 'data/processed'")

Datos exportados correctamente a 'data/processed'


In [65]:
# Ruta y nombre del archivo a guardar.
parquet_path = os.path.join(output_dir, 'pc_desanidado.parquet')
# Guardar el DataFrame en la carpeta data/processed
pc_desanidado.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a 'data/processed'")

Datos exportados correctamente a 'data/processed'


In [66]:
# Ruta y nombre del archivo a guardar.
parquet_path = os.path.join(output_dir, 'pctry_desanidado.parquet')
# Guardar el DataFrame en la carpeta data/processed
pctry_desanidado.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a 'data/processed'")

Datos exportados correctamente a 'data/processed'


In [67]:
# Ruta y nombre del archivo a guardar.
parquet_path = os.path.join(output_dir, 'slan_desanidado.parquet')
# Guardar el DataFrame en la carpeta data/processed
slan_desanidado.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a 'data/processed'")

Datos exportados correctamente a 'data/processed'


# Ahora credits.csv

Transformemos a parquet y leamos el archivo.

In [70]:
from convert_csv_to_parquet import convertir_csv_a_parquet

convertir_csv_a_parquet(
    r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\raw\credits.csv',
    r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\raw\credits.parquet'
)

In [68]:
import pandas as pd

df = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\raw\credits.parquet')
df.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
